### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  

* Lisosia93 is the top spending customer with 5 games totaling $18.96.  

* "Oathbreaker, Last Hope of the Breaking Storm" is the best selling game by sales volume (12) and revenue ($50.76).  

* "Nirvana" and "Fiery Glass Crusader" are the next two top selling games with sales volume of 9 and high revenues. 

* "Nirvana" revenue is $44.10

* "Fiery Glass Crusader" revenue is $41.22  
-----

### Dependencies and Setup
* Import pandas and numpy
* dfheader_remove_upper_space : Function to cleanup column names i.e lowercase with underscores

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# define useful functions
def dfheader_remove_upper_space(df):
    '''
    change column names of dataframe to lowercase with underscores
    '''
    # get list of column names
    oldcol = list(df.columns.values)
    # convert each column name to lowercase and replace space with underscore
    newcol = [s.lower().replace(' ', '_') for s in oldcol]
    # create a rename dict with the above
    rendict = dict(zip(oldcol, newcol))
    # return renamed dataframe
    return(df.rename(columns = rendict))
##end dfheader_remove_upper_space(df)


### Import Data
* Open Purchasing Data File and store into Pandas data frame
* Establish bins for ages and categorize players using the age bins - store in new column in dataframe

In [2]:
# File to Load
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
# cleanup column names
purchase_data = dfheader_remove_upper_space(purchase_data)

# create age group bins in increments of 5 years and bin labels
age_bins = [0,] + list(np.arange(9, purchase_data.age.max(), 5))
age_bins[-1] = purchase_data.age.max()+1
bin_labels = [f'{age_bins[i]+1}-{v}' for (i,v) in enumerate(age_bins[1:])]
bin_labels[0] = '<10'
bin_labels[-1] = '40+'
# add new column with age group info into dataframe
purchase_data['age_group'] = pd.cut(purchase_data.age, age_bins, labels=bin_labels)
purchase_data.head()

,purchase_id,sn,age,gender,item_id,item_name,price,age_group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


### Define Some Useful Functions
* dollar_format : Convert columns with float values to strings with $x.yy format
* total_players : Return Number of Players from dataframe/groupby object
* num_unique_items : Return Number of Items from dataframe/groupby object
* purchase_count : Purchase Count from dataframe/groupby object
* avg_purchase_price : Average Purchase Price from dataframe/groupby object
* total_purchase_value : Total Purchase Value from dataframe/groupby object

In [3]:
def dollar_format(df):
    '''
    Reformats columns with float values to strings with $x.yy format   
    '''
    # identify columns with float data type in the dataframe
    mygen = (c for c in df.columns if df[c].dtypes == 'float64')
    # cast the identified columns to the target format
    for dfcol in mygen:
        df[dfcol] = df[dfcol].map("${:.2f}".format)

    return df
##end dollar_format(df)

def total_players(dfgrp):
    return dfgrp.sn.nunique()
##end total_players(dfgrp)

def num_unique_items(dfgrp):
    return dfgrp.item_id.nunique()
##end num_unique_items(dfgrp)

def purchase_count(dfgrp):
    return dfgrp.purchase_id.count()
##end purchase_count(dfgrp)

def avg_purchase_price(dfgrp):
    return dfgrp.price.mean()
##end avg_purchase_price(dfgrp)

def total_purchase_value(dfgrp):
    return dfgrp.price.sum()
##end total_purchase_value(dfgrp)


## Player Count

* Display the total number of players


In [4]:
pd.DataFrame({'total_players' : total_players(purchase_data)}, index=[0])

,total_players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
# Creating a summary DataFrame using above values
purchasing_analysis = pd.DataFrame({"num_unique_items": num_unique_items(purchase_data),
                                    "avg_price": avg_purchase_price(purchase_data),
                                    "num_purchases": purchase_count(purchase_data),
                                    "total_revenue": total_purchase_value(purchase_data)}, index=[0])
purchasing_analysis = dollar_format(purchasing_analysis)
purchasing_analysis.head()

,num_unique_items,avg_price,num_purchases,total_revenue
0,183,$3.05,780,$2379.77


### Define Useful Functions
* demographic_analysis : Return purchasing analysis summary dataframe with purchase info for players of each type 

In [6]:
def demographic_analysis(df, demo):
    '''
    Demographic analysis by specified column name - gender or age group
    '''
    # group by the requested demographic column
    demo_grp = df.groupby([demo])

    # create a demographic summary dataframe number of players of each type
    demo_count = total_players(demo_grp)
    demo_analysis = pd.DataFrame({'total_count' : demo_count, 
                                  'percentage_of_players' : demo_count*100.0/total_players(df)},
                                 index=demo_count.index)
    # round the percentage column to 2 digits after decimal point
    demo_analysis.percentage_of_players = demo_analysis.percentage_of_players.map("{:.2f}".format)

    # create a purchasing analysis summary dataframe with purchase info for players of each type
    total_purchase_per_person = df.groupby([demo, 'sn']).price.sum()
    purchasing_analysis_demo = pd.DataFrame({"purchase_count": purchase_count(demo_grp),
                                             "avg_purchase_price": avg_purchase_price(demo_grp),
                                             "total_purchase_value": total_purchase_value(demo_grp),
                                             "avg_total_purchase_per_person": total_purchase_per_person.mean(level=demo)})
    purchasing_analysis_demo = dollar_format(purchasing_analysis_demo)

    return (demo_analysis, purchasing_analysis_demo)
##end demographic_analysis(demo)


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [7]:
(gender_demo, purchasing_analysis_gender) = demographic_analysis(purchase_data, 'gender')
gender_demo

,total_count,percentage_of_players
gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
purchasing_analysis_gender

,purchase_count,avg_purchase_price,total_purchase_value,avg_total_purchase_per_person
gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [9]:
(age_demo, purchasing_analysis_age) = demographic_analysis(purchase_data, 'age_group')
age_demo

,total_count,percentage_of_players
age_group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [10]:
purchasing_analysis_age

,purchase_count,avg_purchase_price,total_purchase_value,avg_total_purchase_per_person
age_group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


### Define Useful Functions
* top_grp_analysis : Return purchasing analysis summary dataframe with purchase info for group of each type sorted by given metric

In [11]:
def top_grp_analysis(df, collist, metric, pricecol):
    '''
    Purchasing analysis by specified columns based on given metric
    '''
    # group by the requested column list
    interested_grp = df.groupby(collist)
    # create a purchasing analysis summary dataframe with purchase info for group of each type
    grp_purchasing_analysis = pd.DataFrame({"purchase_count": purchase_count(interested_grp),
                                            pricecol: avg_purchase_price(interested_grp),
                                            "total_purchase_value": total_purchase_value(interested_grp)})
    # sort rows in descending order by metric
    grp_purchasing_analysis = grp_purchasing_analysis.sort_values(by=metric, ascending=False)
    return(dollar_format(grp_purchasing_analysis))
##end top_grp_analysis(collist, metric)
    

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
top_grp_analysis(purchase_data, ['sn'], 'total_purchase_value', 'avg_purchase_price').head()

,purchase_count,avg_purchase_price,total_purchase_value
sn,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
top_grp_analysis(purchase_data, ['item_id', 'item_name'], 'purchase_count', 'item_price').head()

,,purchase_count,item_price,total_purchase_value
item_id,item_name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [14]:
top_grp_analysis(purchase_data, ['item_id', 'item_name'], 'total_purchase_value', 'item_price').head()

,,purchase_count,item_price,total_purchase_value
item_id,item_name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
